In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.ensemble import RandomForestClassifier

df = pd.read_csv('data/train.csv')
df['Sex'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 891 entries, 0 to 890
Series name: Sex
Non-Null Count  Dtype 
--------------  ----- 
891 non-null    object
dtypes: object(1)
memory usage: 7.1+ KB


In [12]:
cond1 = df ['Sex'] == 'female'
cond2 = df ['Sex'] == 'male'
df.loc[cond1, 'Age'] = df.loc[cond1, 'Age'].fillna(df.loc[cond1, 'Age'].mean())
df.loc[cond2, 'Age'] = df.loc[cond2, 'Age'].fillna(df.loc[cond2, 'Age'].mean())

df = pd.get_dummies(df, columns=['Sex'], dtype=int)

data = df[['PassengerId', 'Pclass', 'Sex_female', 'Age', 'SibSp', 'Parch', 'Fare']]
target = df[['Survived']]

In [16]:
df.head()

,PassengerId,Survived,Pclass,Name,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Sex_female,Sex_male
0,1,0,3,"Braund, Mr. Owen Harris",22.0,1,0,A/5 21171,7.2500,NaN,S,0,1
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",38.0,1,0,PC 17599,71.2833,C85,C,1,0
2,3,1,3,"Heikkinen, Miss. Laina",26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1,0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",35.0,1,0,113803,53.1000,C123,S,1,0
4,5,0,3,"Allen, Mr. William Henry",35.0,0,0,373450,8.0500,NaN,S,0,1


In [13]:
train_input, test_input, train_target, test_target = train_test_split(data, target)

poly = PolynomialFeatures(degree=2, include_bias=False)
train_poly = poly.fit_transform(train_input)
test_poly = poly.transform(test_input)

# lr = LinearRegression()
# lr.fit(train_poly, train_target)

ss = StandardScaler()
train_scaled = ss.fit_transform(train_poly)
test_scaled = ss.transform(test_poly)

In [14]:
sub_df = pd.read_csv('data/test.csv')

cond1 = sub_df ['Sex'] == 'female'
cond2 = sub_df ['Sex'] == 'male'
sub_df.loc[cond1, 'Age'] = sub_df.loc[cond1, 'Age'].fillna(sub_df.loc[cond1, 'Age'].mean())
sub_df.loc[cond2, 'Age'] = sub_df.loc[cond2, 'Age'].fillna(sub_df.loc[cond2, 'Age'].mean())

sub_df.loc[:, 'Age'] = sub_df['Age'].fillna(sub_df['Age'].mean())
sub_df.loc[:, 'Fare'] = sub_df['Fare'].fillna(sub_df['Fare'].mean())

sub_df = pd.get_dummies(sub_df, columns=['Sex'], dtype=int)

In [21]:
sub_df[['Sex_female']].head()

,Sex_female
0,0
1,1
2,0
3,0
4,1


In [15]:
sub_data = sub_df[['PassengerId', 'Pclass', 'Sex_female', 'Age', 'SibSp', 'Parch', 'Fare']]

sub_poly = poly.transform(sub_data)
sub_scaled = ss.transform(sub_poly)

rf = RandomForestClassifier()
rf.fit(train_scaled, train_target.values.ravel())
sub_predictions = rf.predict(sub_scaled)

sub_df.insert(1, 'Survived', sub_predictions)
sub_df[['PassengerId', 'Survived']]

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0
